In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Get train data
train = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv")
print("train : " + str(train.shape))

In [ ]:
train.head()

In [ ]:
# Get test data
test = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/test.csv")
print("test : " + str(test.shape))

In [ ]:
import math, datetime

import numpy as np 
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import(mean_squared_error)

In [ ]:
# Convert date values to datetime
train['date'] = pd.to_datetime(train['date'])

In [ ]:
# Convert a datetime column to a string one
train['year_month'] = train['date'].apply(lambda x: x.strftime('%Y-%m')) 

In [ ]:
# Drop unnecessary features
train = train.drop(['date','item_price'], axis=1)
train.head(3)

In [ ]:
# group features to get the number of products sold per month. You are predicting a monthly amount of this measure
train_group = train.groupby(['year_month', 'shop_id', 'item_id']).sum().reset_index()
train_group.head()

In [ ]:
df = train_group.pivot_table(index=['shop_id','item_id'], columns='year_month', values='item_cnt_day', 
                        fill_value=0)
df.reset_index(inplace=True)
df.head()

In [ ]:
test_id = test['ID']

In [ ]:
df_test = pd.merge(test, df, on=['shop_id','item_id'], how='left')
df_test.drop(['ID','2013-01'], axis=1, inplace=True)
df_test = df_test.fillna(0)
df_test.head()

In [ ]:
X = df[df.columns[:-1]]
y = df[df.columns[-1]]
print(X)
print(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, test_size=0.2, random_state=15)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
regressor.fit(X,y)

In [ ]:
sub = pd.DataFrame()


# RandomForestRegressor
prediction = regressor.predict(df_test)
prediction = list(map(round, prediction))
sub['Id'] = test_id
sub['item_cnt_month'] = prediction
sub.to_csv('mysubmission.csv',index=False)
print(sub)
